In [8]:
import numpy as np
import math

def read_tsp_file(file_path):
    city_coordinates = {}
    with open(file_path, 'r') as file:
        # Temukan awal dari NODE_COORD_SECTION
        for line in file:
            if line.strip() == "NODE_COORD_SECTION":
                break
        
        # Baca data koordinat kota
        for line in file:
            if line.strip() == "EOF":
                break
            # Split line and convert values to integers
            values = list(map(int, line.split()))
            
            if len(values) == 2:
                # If there are two values, assume (x, y) and use the line number as city_id
                city_id = len(city_coordinates) + 1
                x, y = values
            elif len(values) == 3:
                # If there are three values, assume (city_id, x, y)
                city_id, x, y = values
            else:
                # Handle other cases as needed
                continue
            
            city_coordinates[city_id] = (x, y)
    
    return city_coordinates

file_path = 'D:/att48.tsp'
city_coordinates = read_tsp_file(file_path)

# Function to calculate Euclidean distance
def euclidean_distance(city1, city2):
    return math.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

# Hasilkan populasi awal menggunakan metode acak
def generate_initial_population(city_coordinates, population_size):
    city_ids = list(city_coordinates.keys())
    population = [np.random.permutation(city_ids).tolist() for _ in range(population_size)]
    return population

# Hitung total jarak suatu jalur
def calculate_total_distance(path, city_coordinates):
    total_distance = 0
    for i in range(len(path)):
        from_city = city_coordinates[path[i]]
        to_city = city_coordinates[path[(i + 1) % len(path)]]  # Loop back to start
        total_distance += euclidean_distance(from_city, to_city)
    return total_distance

# Fitness evaluation
def evaluate_fitness(population, city_coordinates):
    fitness_scores = []
    for individual in population:
        distance = calculate_total_distance(individual, city_coordinates)
        fitness_score = 1 / distance  # Higher fitness for shorter path
        fitness_scores.append(fitness_score)
    return np.array(fitness_scores)

# Selection - Roulette Wheel Selection
def select_parents(population, fitness_scores, num_parents):
    parents = []
    for _ in range(num_parents):
        parent_idx = np.random.choice(np.arange(len(population)), p=fitness_scores/fitness_scores.sum())
        parents.append(population[parent_idx])
    return parents

def two_point_crossover(parent1, parent2):
    size = len(parent1)
    # Hasilkan dua titik berbeda untuk saling bersilangan
    crossover_points = sorted(np.random.choice(range(1, size), 2, replace=False))
    start, end = crossover_points
    
    # Initialize the child with None
    """
    menginisialisasi daftar bernama child dengan panjang sama dengan size, dimana setiap elemen dalam daftar awalnya disetel ke None.
    Menginisialisasi [None] dapat membantu memeriksa error. 
    Setelah operasi crossover selesai, jika ada nilai None yang tersisa, hal ini menunjukkan bahwa algoritma 
    tidak mengisi array child dengan benar, yang dapat membantu dalam men-debug proses crossover.
    """
    child = [None] * size
    
    # Salin segmen dari parent1 ke anak
    child[start:end] = parent1[start:end]
    # Isi sisa rute dari induk2, pastikan tidak ada duplikat
    current_position = end
    for city in parent2:
        if city not in child:
            if current_position >= size:
                current_position = 0
            child[current_position] = city
            current_position += 1
    
    return child


# Mutation - Swap Mutation
def swap_mutation(individual, mutation_rate):
    if np.random.rand() < mutation_rate:
        idx1, idx2 = np.random.choice(len(individual), 2, replace=False)
        individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual

# Jalankan algoritma genetika untuk TSP
def genetic_algorithm_tsp_with_elitism(city_coordinates, population_size=10, num_generations=100, mutation_rate=0.01, elitism_size=2):
    population = generate_initial_population(city_coordinates, population_size)
    for generation in range(num_generations):
        
        #Evaluasi Fitness
        fitness_scores = evaluate_fitness(population, city_coordinates)
        total_fitness = np.sum(fitness_scores)
        best_fitness = np.max(fitness_scores)
        best_distance = 1 / np.max(fitness_scores)
        
        # Elitisme: Cadangan individu terbaik dari populasi saat ini
        elite_indices = np.argsort(-fitness_scores)[:elitism_size]
        elites = [population[i] for i in elite_indices]
        
        # Hasilkan populasi berikutnya, tidak termasuk elit untuk saat ini
        next_population = []
        parents = select_parents(population, fitness_scores, population_size - elitism_size)
        for i in range(0, len(parents), 2):
            if i+1 < len(parents):  # Ensure there's a pair for crossover
                for child in [two_point_crossover(parents[i], parents[i+1]), two_point_crossover(parents[i+1], parents[i])]:
                    child = swap_mutation(child, mutation_rate)
                    next_population.append(child)
        
        # Tambahkan elit ke populasi berikutnya setelah mutasi
        next_population.extend(elites)
        
        population = next_population
        
        # Perbarui skor kebugaran berdasarkan populasi baru termasuk elit
        fitness_scores = evaluate_fitness(population, city_coordinates)
        best_distance = 1 / np.max(fitness_scores)  # Hitung ulang jarak terbaik dengan menyertakan elit
        best_fitness = np.max(fitness_scores)
        
        # Display generation info      
        print(f"Generasi {generation + 1}\nBest Distance = {best_distance}\nNilai Fitness = {best_fitness}")
        # print(f"Total Fitness = {total_fitness}")
        # Display best path in current generation
        best_idx = np.argmax(fitness_scores)
        best_path = population[best_idx]
        print(f"Rute: {best_path}\n")


    # Identifikasi solusi terbaik dalam populasi akhir
    best_idx = np.argmax(fitness_scores)
    best_solution = population[best_idx]
    return best_solution, best_distance

# Jalankan Algoritma Genetika dengan elitisme dan tampilkan hasilnya
best_path, best_distance = genetic_algorithm_tsp_with_elitism(city_coordinates)
print("------------------------------------------------------------------------------------------------------------")
print(f"\nBest Distance (All Generation): {best_distance}\nBest Path:{best_path}\n")

Generasi 1
Best Distance = 140612.93031971605
Nilai Fitness = 7.111721501900775e-06
Rute: [8, 28, 37, 44, 20, 47, 10, 34, 39, 36, 2, 25, 19, 17, 46, 6, 22, 3, 11, 38, 40, 21, 4, 30, 31, 9, 26, 42, 14, 15, 7, 43, 13, 16, 35, 27, 32, 23, 5, 1, 41, 29, 24, 12, 18, 33, 45, 48]

Generasi 2
Best Distance = 140612.93031971605
Nilai Fitness = 7.111721501900775e-06
Rute: [8, 28, 37, 44, 20, 47, 10, 34, 39, 36, 2, 25, 19, 17, 46, 6, 22, 3, 11, 38, 40, 21, 4, 30, 31, 9, 26, 42, 14, 15, 7, 43, 13, 16, 35, 27, 32, 23, 5, 1, 41, 29, 24, 12, 18, 33, 45, 48]

Generasi 3
Best Distance = 140612.93031971605
Nilai Fitness = 7.111721501900775e-06
Rute: [8, 28, 37, 44, 20, 47, 10, 34, 39, 36, 2, 25, 19, 17, 46, 6, 22, 3, 11, 38, 40, 21, 4, 30, 31, 9, 26, 42, 14, 15, 7, 43, 13, 16, 35, 27, 32, 23, 5, 1, 41, 29, 24, 12, 18, 33, 45, 48]

Generasi 4
Best Distance = 140612.93031971605
Nilai Fitness = 7.111721501900775e-06
Rute: [8, 28, 37, 44, 20, 47, 10, 34, 39, 36, 2, 25, 19, 17, 46, 6, 22, 3, 11, 38, 40, 21, 

Generasi 61
Best Distance = 101737.58495617945
Nilai Fitness = 9.829209140660468e-06
Rute: [41, 34, 5, 10, 42, 35, 2, 4, 26, 24, 25, 17, 19, 47, 11, 14, 38, 40, 13, 6, 33, 22, 12, 45, 46, 7, 43, 28, 37, 44, 8, 1, 20, 21, 36, 39, 15, 18, 27, 32, 30, 31, 9, 23, 16, 29, 48, 3]

Generasi 62
Best Distance = 97670.58219077562
Nilai Fitness = 1.0238497381399287e-05
Rute: [44, 8, 1, 20, 42, 35, 2, 4, 26, 24, 25, 17, 19, 47, 11, 14, 38, 40, 13, 21, 36, 39, 15, 30, 18, 27, 32, 29, 23, 16, 3, 41, 6, 33, 22, 12, 31, 9, 34, 5, 10, 45, 48, 46, 7, 43, 28, 37]

Generasi 63
Best Distance = 97670.58219077562
Nilai Fitness = 1.0238497381399287e-05
Rute: [44, 8, 1, 20, 42, 35, 2, 4, 26, 24, 25, 17, 19, 47, 11, 14, 38, 40, 13, 21, 36, 39, 15, 30, 18, 27, 32, 29, 23, 16, 3, 41, 6, 33, 22, 12, 31, 9, 34, 5, 10, 45, 48, 46, 7, 43, 28, 37]

Generasi 64
Best Distance = 97670.58219077562
Nilai Fitness = 1.0238497381399287e-05
Rute: [44, 8, 1, 20, 42, 35, 2, 4, 26, 24, 25, 17, 19, 47, 11, 14, 38, 40, 13, 21, 36, 

In [26]:
# # Loop through all pairs of cities and calculate distances
# for city1_id, city1_coords in city_coordinates.items():
#     for city2_id, city2_coords in city_coordinates.items():
#         if city1_id != city2_id:
#             distance = euclidean_distance(city1_coords, city2_coords)
#             rounded_distance = round(distance)  # Menyimpan nilai yang sudah dibulatkan
#             print(f"Euclidean distance between city {city1_id} and city {city2_id}: {rounded_distance}")

In [18]:
# import numpy as np
# import math
# import time

# def read_tsp_file(file_path):
#     city_coordinates = {}
#     with open(file_path, 'r') as file:
#         # Temukan awal dari NODE_COORD_SECTION
#         for line in file:
#             if line.strip() == "NODE_COORD_SECTION":
#                 break
        
#         # Baca data koordinat kota
#         for line in file:
#             if line.strip() == "EOF":
#                 break
#             # Split line and convert values to integers
#             values = list(map(int, line.split()))
            
#             if len(values) == 2:
#                 # If there are two values, assume (x, y) and use the line number as city_id
#                 city_id = len(city_coordinates) + 1
#                 x, y = values
#             elif len(values) == 3:
#                 # If there are three values, assume (city_id, x, y)
#                 city_id, x, y = values
#             else:
#                 # Handle other cases as needed
#                 continue
            
#             city_coordinates[city_id] = (x, y)
    
#     return city_coordinates

# file_path = 'D:/att48.tsp'
# city_coordinates = read_tsp_file(file_path)

# # Function to calculate Euclidean distance
# def euclidean_distance(city1, city2):
#     return math.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

# # Hasilkan populasi awal menggunakan metode acak
# def generate_initial_population(city_coordinates, population_size):
#     city_ids = list(city_coordinates.keys())
#     population = [np.random.permutation(city_ids).tolist() for _ in range(population_size)]
#     return population

# # Hitung total jarak suatu jalur
# def calculate_total_distance(path, city_coordinates):
#     total_distance = 0
#     for i in range(len(path)):
#         from_city = city_coordinates[path[i]]
#         to_city = city_coordinates[path[(i + 1) % len(path)]]  # Loop back to start
#         total_distance += euclidean_distance(from_city, to_city)
#     return total_distance

# # Fitness evaluation
# def evaluate_fitness(population, city_coordinates):
#     fitness_scores = []
#     for individual in population:
#         distance = calculate_total_distance(individual, city_coordinates)
#         fitness_score = 1 / distance  # Higher fitness for shorter path
#         fitness_scores.append(fitness_score)
#     return np.array(fitness_scores)

# # Selection - Roulette Wheel Selection
# def select_parents(population, fitness_scores, num_parents):
#     parents = []
#     for _ in range(num_parents):
#         parent_idx = np.random.choice(np.arange(len(population)), p=fitness_scores/fitness_scores.sum())
#         parents.append(population[parent_idx])
#     return parents

# def two_point_crossover(parent1, parent2):
#     size = len(parent1)
#     # Hasilkan dua titik berbeda untuk saling bersilangan
#     crossover_points = sorted(np.random.choice(range(1, size), 2, replace=False))
#     start, end = crossover_points
    
#     # Initialize the child with None
#     """
#     menginisialisasi daftar bernama child dengan panjang sama dengan size, dimana setiap elemen dalam daftar awalnya disetel ke None.
#     Menginisialisasi [None] dapat membantu memeriksa error. 
#     Setelah operasi crossover selesai, jika ada nilai None yang tersisa, hal ini menunjukkan bahwa algoritma 
#     tidak mengisi array child dengan benar, yang dapat membantu dalam men-debug proses crossover.
#     """
#     child = [None] * size
    
#     # Salin segmen dari parent1 ke anak
#     child[start:end] = parent1[start:end]
#     # Isi sisa rute dari induk2, pastikan tidak ada duplikat
#     current_position = end
#     for city in parent2:
#         if city not in child:
#             if current_position >= size:
#                 current_position = 0
#             child[current_position] = city
#             current_position += 1
    
#     return child


# # Mutation - Swap Mutation
# def swap_mutation(individual, mutation_rate):
#     if np.random.rand() < mutation_rate:
#         idx1, idx2 = np.random.choice(len(individual), 2, replace=False)
#         individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
#     return individual

# # Jalankan algoritma genetika untuk TSP
# def genetic_algorithm_tsp_with_elitism(city_coordinates, population_size=10, num_generations=100, mutation_rate=0.01, elitism_size=2):
#     start_time = time.time() #start timing the algorithm 
    
#     population = generate_initial_population(city_coordinates, population_size)
#     for generation in range(num_generations):
#         #Evaluasi Fitness
#         fitness_scores = evaluate_fitness(population, city_coordinates)
#         total_fitness = np.sum(fitness_scores)
#         best_distance = 1 / np.max(fitness_scores)
        
#         # Elitisme: Cadangan individu terbaik dari populasi saat ini
#         elite_indices = np.argsort(-fitness_scores)[:elitism_size]
#         elites = [population[i] for i in elite_indices]
        
#         # Hasilkan populasi berikutnya, tidak termasuk elit untuk saat ini
#         next_population = []
#         parents = select_parents(population, fitness_scores, population_size - elitism_size)
#         for i in range(0, len(parents), 2):
#             if i+1 < len(parents):  # Ensure there's a pair for crossover
#                 for child in [two_point_crossover(parents[i], parents[i+1]), two_point_crossover(parents[i+1], parents[i])]:
#                     child = swap_mutation(child, mutation_rate)
#                     next_population.append(child)
        
#         # Tambahkan elit ke populasi berikutnya setelah mutasi
#         next_population.extend(elites)
        
#         population = next_population
        
#         # Perbarui skor kebugaran berdasarkan populasi baru termasuk elit
#         fitness_scores = evaluate_fitness(population, city_coordinates)
#         best_distance = 1 / np.max(fitness_scores)  # Hitung ulang jarak terbaik dengan menyertakan elit
        
#         # Display generation info      
#         print(f"Generasi {generation + 1}\nBest Distance = {best_distance}\nTotal Fitness = {total_fitness}")
        
#         # Display best path in current generation
#         best_idx = np.argmax(fitness_scores)
#         best_path = population[best_idx]
#         print(f"Rute: {best_path}\n")
    
#     computation_time = time.time() - start_time
        

#     # Identifikasi solusi terbaik dalam populasi akhir
#     best_idx = np.argmax(fitness_scores)
#     best_solution = population[best_idx]
# #     best_distance = 1 / fitness_scores[best_idx]
#     return best_solution, best_distance, computation_time

# # Jalankan Algoritma Genetika dengan elitisme dan tampilkan hasilnya
# best_path, best_distance, computation_time = genetic_algorithm_tsp_with_elitism(city_coordinates)
# print("------------------------------------------------------------------------------------------------------------")
# print(f"\nComputation Time: {computation_time:.4f} seconds")
# print(f"\nBest Path:{best_path}\nBest Distance: {best_distance}")